# Fetching National Water Model Streamflow Forecasts

## Overview
In this guide we'll demonstrate fetching National Water Model (NWM) streamflow forecasts from Google Cloud Storage (GCS). This example makes use of a pre-generated Evaluation dataset stored in the public S3 bucket containing a single USGS gage location and the corresponding NWM location ID.

For a refresher on loading location and location crosswalk data into a new Evaluation refer back to the [](/user_guide/notebooks/02_loading_local_data.ipynb) and [](/user_guide/notebooks/04_setup_simple_example.ipynb) user guide pages.

### Clone the example Evaluation

In [9]:
from datetime import datetime
from pathlib import Path
import shutil

import teehr

# Tell Bokeh to output plots in the notebook
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [15]:
# Define the directory where the Evaluation will be created
test_eval_dir = Path(Path().home(), "temp", "10_fetch_nwm_data")
shutil.rmtree(test_eval_dir, ignore_errors=True)

# Create an Evaluation object and create the directory
ev = teehr.Evaluation(dir_path=test_eval_dir, create_dir=True)

In [16]:
# List the evaluations in the S3 bucket
ev.list_s3_evaluations()

,name,description,url
0,e0_2_location_example,Example evaluation datsets with 2 USGS gages,s3a://ciroh-rti-public-data/teehr-data-warehou...
1,e1_camels_daily_streamflow,Daily average streamflow at ther Camels basins,s3a://ciroh-rti-public-data/teehr-data-warehou...
2,e2_camels_hourly_streamflow,Hourly instantaneous streamflow at ther Camels...,s3a://ciroh-rti-public-data/teehr-data-warehou...
3,e3_usgs_hourly_streamflow,Hourly instantaneous streamflow at USGS CONUS ...,s3a://ciroh-rti-public-data/teehr-data-warehou...
4,e4_hefs_ensemble_example,An example with HEFS ensemble streamflow at 2 ...,s3a://ciroh-rti-public-data/teehr-data-warehou...
5,e5_nwm_streamflow_fetching_example,An example with a single location to demonstra...,s3a://ciroh-rti-public-data/teehr-data-warehou...


In [17]:
# Clone the e5_nwm_streamflow_fetching_example evaluation from the S3 bucket
ev.clone_from_s3("e5_nwm_streamflow_fetching_example")

AnalysisException: [UNABLE_TO_INFER_SCHEMA] Unable to infer schema for Parquet. It must be specified manually.

This example Evaluation only contains a single location, the USGS gage on the New River at Radford, VA.

In [5]:
locations_gdf = ev.locations.to_geopandas()
locations_gdf

,id,name,geometry
0,usgs-03171000,"NEW RIVER AT RADFORD, VA",POINT (-80.56922 37.14179)


The ID of the National Water Model reach corresponding to the USGS gage is in the `location crosswalks` table.

In [ ]:
location_crosswalks_df = ev.location_crosswalks.to_pandas()
location_crosswalks_df

,primary_location_id,secondary_location_id
0,usgs-03171000,nwm30-6884666


### Fetching USGS streamgage data

Since the example Evalution already contains the location and location crosswalk IDs for USGS and NWM locations, we can make use of TEEHR's built-in tools to fetch streamflow data for each location.

First we'll fetch USGS data from the National Water Information System (NWIS). TEEHR makes use of the USGS [dataretrieval](https://github.com/DOI-USGS/dataretrieval-python) python tool under the hood.

In [ ]:
ev.fetch.usgs_streamflow()

In [ ]:
ev.fetch.usgs_streamflow(
    start_date=datetime(2024, 9, 26),
    end_date=datetime(2024, 10, 1)
)